<a href="https://colab.research.google.com/github/Buchunwang/UK-CLI/blob/main/UK_CLI_kalman_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we import detrended data of CLI and its component variables from 1986 to 2021

In [1]:
import os
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/Buchunwang/UK-CLI/main/kalpre.csv'
print('From URL:', url)
kalpre = pd.read_csv(url, header=None, encoding='utf-8', skiprows=1)

From URL: https://raw.githubusercontent.com/Buchunwang/UK-CLI/main/kalpre.csv


Then we standardize the data

In [2]:
from sklearn.preprocessing import StandardScaler
scale= StandardScaler()
kalpre = scale.fit_transform(kalpre) 
print(kalpre)

[[ 4.77728394e-02 -1.06056869e-02  3.00724295e-03 ... -1.03166195e-01
   8.00207365e-03 -6.11869099e-02]
 [ 9.88213626e-02  5.18563334e-02 -7.53982191e-03 ...  3.85044329e-01
   1.64688170e+00  1.85709938e+00]
 [ 7.23062404e-02  1.03068101e-01  5.50476655e-02 ...  3.08487391e+00
  -1.26525437e+00  1.62538219e+00]
 ...
 [-2.48640418e-01  1.30844793e+00 -2.29908393e-01 ... -6.93214447e-02
  -2.77564022e+00  3.46200192e+00]
 [ 1.28896015e+00 -2.45504802e-01  1.31416337e+00 ... -6.93214447e-02
   4.81092770e-01  8.82828354e-01]
 [ 1.63507006e+00  1.29701274e+00 -2.42910777e-01 ... -1.03166195e-01
  -2.63429707e+00 -5.11759895e-01]]


In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression

N = 421
Y = np.transpose(kalpre[:,3:9])
lamda = np.zeros((1, 5))
D = np.zeros((1, 5))
var_e = np.zeros((1, 5))
var_ep = np.zeros((1, 5))
xe = np.zeros((N - 2, 2))
psi = np.zeros((5, 2))

for i in range(5):
    reg = LinearRegression(fit_intercept=True)
    reg.fit(kalpre[:, 0:1], Y[i, :])
    lamda[0, i] = reg.coef_[0]
    D[0, i] = reg.intercept_
    r = Y[i, :] - reg.predict(kalpre[:, 0:1])
    var_e[0, i] = np.var(r)
    
    xe[:, 0] = r[1:N-1]
    xe[:, 1] = r[0:N-2]
    reg2 = LinearRegression(fit_intercept=False)
    reg2.fit(xe, r[2:N])
    var_ep[0, i] = np.var(reg2.predict(xe) - r[2:N])
    psi[i, 0] = reg2.coef_[0]
    psi[i, 1] = reg2.coef_[1]


In [56]:
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
# Calculate delta_y, delta_y_t-1, and delta_y_t-2
y = np.zeros((5, N))
y1lag = np.zeros((5, N))
y2lag = np.zeros((5, N))
for i in range(5):
    y[i-1, :] = kalpre[:, i + 2] - np.mean(kalpre[:, i + 2])
    y1lag[i-1, :] = kalpre[:, i + 7] - np.mean(kalpre[:, i + 7])
    y2lag[i-1, :] = kalpre[:, i + 12] - np.mean(kalpre[:, i + 12])

# Calculate phi
# Calculate phi based on equation 3 of Kim and Nelson
regressor = kalpre[:, 1:3] - np.mean(kalpre[:, 1:3], axis=0)  # 2nd and 3rd columns of kalpre
response = kalpre[:, 0] - np.mean(kalpre[:, 0])  # 1st column of kalpre
regressor = sm.add_constant(regressor)
model = sm.OLS(response, regressor)
result = model.fit()
phi = result.params
var_vt = np.var(result.resid)
mu_st = kalpre[:, 0] - phi[0] * kalpre[:, 1] - phi[1] * kalpre[:, 2] - np.mean(kalpre[:, 0]) - \
        norm.rvs(loc=0, scale=np.sqrt(var_vt), size=N,)
mu_st = mu_st.reshape((1, -1))


# Construct H and A
H = np.array([[lamda[0,0], -lamda[0,0]*psi[0, 0], -lamda[0,0]*psi[0, 1]],
              [lamda[0,1], -lamda[0,1]*psi[1, 0], -lamda[0,1]*psi[1, 1]],
              [lamda[0,2], -lamda[0,2]*psi[2, 0], -lamda[0,2]*psi[2, 1]],
              [lamda[0,3], -lamda[0,3]*psi[3, 0], -lamda[0,3]*psi[3, 1]],
              [lamda[0,4], -lamda[0,4]*psi[4, 0], -lamda[0,4]*psi[4, 1]]])
A = np.array([[phi[0], phi[1], 0],
              [0, phi[0], phi[1]],
              [0, 1, 0]])

# Construct delta_y*
ystar = y.T - np.multiply(psi[:, 0].T, y1lag[0:5, :].T)- np.multiply(psi[:, 1].T,y2lag[0:5, :].T)
ystar=ystar.T


# Initialize variables
Q = np.array([[var_vt, 0, 0],
[0, var_vt, 0],
[0, 0, 0]])
R = np.array([[var_ep[0,0], 0, 0, 0, 0],
             [0, var_ep[0,1], 0, 0, 0],
             [0, 0, var_ep[0,2], 0, 0],
             [0, 0, 0, var_ep[0,3], 0],
             [0, 0, 0, 0, var_ep[0,4]]])
P = 0.1 * np.eye(3)
z = np.ones((5, N))
k = 0
true5D = ystar
x = np.zeros((3, N))
vct = np.zeros((3, N))


In [91]:
k = 0
for t in range(N-1):
    k += 1
    x[:, k] = np.dot(A, x[:, k-1])  # + mut[:, k]  # Kalman equation 1
    P = np.dot(A, np.dot(P, A.T)) + Q  # Kalman equation 2
    K = np.dot(P, np.dot(H.T, np.linalg.inv(np.dot(H, np.dot(P, H.T)) + R)))  # Kalman equation 3
    z[:, k] = [true5D[0, k], true5D[1, k], true5D[2, k], true5D[3, k], true5D[4, k]]   
    x[:, k] = x[:, k] + np.dot(K, (z[:, k] - np.dot(H, x[:, k])))  # Kalman equation 4
    P = np.dot((np.eye(3) - np.dot(K, H)), P)  # Kalman equation 5
    vct[0, k] = P[0, 0]  # save variance of delta_ct
    vct[1, k] = P[1, 1]
    vct[2, k] = P[2, 2]

step2

In [100]:
from numpy.random import normal

c = normal(x[0, N-1], np.sqrt(vct[0, N-1]))  # generate delta_c_T
xi = np.zeros((1, N))
V = np.zeros((1, N))
for i in range(N-1):  # this loop is just for xi_t and V_t conditional on delta_ct+1 for t=1,2,...,T-1
    eta = c - mu_st[0,N-i-1] - phi[0]*x[0, N-i-2] - phi[1]*x[1, N-i-2]  # value of eta_t two lag in my model, that's why I have term -phi(2)*x(2,N-i)
    Rt = np.sum(A[0]*vct[:, N-i-1]*A[0])+var_vt
    xi[0, N-i-1] = x[0, N-i-2] + phi[0]*vct[0, N-i-1]*eta/Rt
    # we only use first row of the mean and variance vector
    # so I calculated the first element of each mean & variance directly
    V[0, N-i-1] = vct[0, N-i-1] - phi[0]**2 * vct[0,N-i-1]**2 / Rt
    c = normal(xi[0, N-i-1], np.sqrt(V[0, N-i-1]))
V[0, N-1] = vct[0, N-1]
xi[0, N-1] = x[0, N-1]
delta=0.0022
delta_C=np.zeros((N,1))
C0=99.72203

In [117]:
import numpy as np
from numpy.random import normal

newCLI = np.zeros((N, 1))
p = 1000
mc = np.zeros((p, 1))
for m in range(p):
    mc[m] = C0 + normal(xi[0,0], np.sqrt(V[0,0])) + delta
newCLI[0] = np.mean(mc)
mc = np.zeros((p, 1))
for i in range(1, N):
    for m in range(p):
        mc[m] = newCLI[i-1] + normal(xi[0,i], np.sqrt(V[0,i]))
    newCLI[i] = np.mean(mc) # new delta_C_t=delta_c_t+C_{t-1}+delta according to appendix A7

cc = np.zeros((N, 1))
mc = np.zeros((p, 1))
for m in range(p):
    mc[m] = delta + C0 + normal(x[0, 0], np.sqrt(vct[0, 0]))
cc[0] = np.mean(mc)
for i in range(1, N):
    for m in range(p):
        mc[m] = cc[i-1] + normal(x[0, i], np.sqrt(vct[0, i]))
    cc[i] = np.mean(mc)

We have finished the Kalman filter part, then we come to the Hamilton Basic filter